In [18]:
##### EXERCICE CHAPITRE 3 #####
### PARTIE 1 ###

#1: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

import pymongo
import tqdm
import numpy as np

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["TODO"]
collection = db["CRUB_exercise"]

lecun_paper = {
    "title": "Deep Learning",
    "authors": {
        "Yann LeCun": {"affiliations": ["Facebook AI Research", "New York University"]},
        "Yoshua Bengio": {"affiliations": ["Department of Computer Science and Operations Research Université de Montréal"]},
        "Geoffrey Hinton": {"affiliations": ["Google", "Department of Computer Science, University of Toronto"]}
    }
}
goodfellow_paper = {
    "title": "Generative Adversarial Nets",
    "authors": {
        "Ian Goodfellow": {"affiliations": ["Universite de Montreal"]},
        "Jean Pouget-Abadie": {"affiliations": ["Ecole Polytechnique"]},
        "Mehdi Mirza": {"affiliations": ["Université de Montréal"]},
        "Bing Xu": {"affiliations": ["Université de Montréal"]},
        "David Warde-Farley": {"affiliations": ["Université de Montréal"]},
        "Sherjil Ozair": {"affiliations": ["Indian Institute of Technology Delhi"]},
        "Aaron Courville": {"affiliations": ["Université de Montréal"]},
        "Yoshua Bengio": {"affiliations": ["CIFAR Senior Fellow"]}
    }
}
papers_dict = {
    "LeCun et al.": lecun_paper,
    "Goodfellow et al.": goodfellow_paper
}

collection.insert_one(papers_dict)


InsertOneResult(ObjectId('662a566e33b7e98b2b0ce791'), acknowledged=True)

In [2]:
#2 : Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

for i in range(1, 4):
    collection.insert_one({"key": "x", "value": 1})

# Afficher les documents avant la suppression
print("Documents avant la suppression :")
for doc in collection.find({"key": "x"}):
    print(doc)

# Supprimer un des documents
collection.delete_one({"key": "x"})

# Afficher les documents après la suppression
print("\nDocuments après la suppression :")
for doc in collection.find({"key": "x"}):
    print(doc)


docs = list(collection.find({"key": "x"}).sort("_id", pymongo.ASCENDING))
if len(docs) == 2:
    if docs[0]["_id"] < docs[1]["_id"]:
        print("\nLe document supprimé était le plus ancien.")
    else:
        print("\nLe document supprimé était le plus récent.")
else:
    print("\nIl y a eu une erreur dans la suppression des documents.")

collection.update_many({"key": "x"}, {"$set": {"value": 4}})

print("\nDocuments après l'incrémentation de la valeur de x :")
for doc in collection.find({"key": "x"}):
    print(doc)


Documents avant la suppression :
{'_id': ObjectId('662a2773b24ef03be462b554'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda542'), 'key': 'x', 'value': 1}

Documents après la suppression :
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda542'), 'key': 'x', 'value': 1}

Il y a eu une erreur dans la suppression des documents.

Documents après l'incrémentation de la valeur de x :
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key':

In [3]:
# Le document supprimé est le plus ancien 

In [17]:
#3: Insert the dict created in the TODO 6 Chapter I in the example collection.
###################  REFAIRE ##########################################

import pymongo
import json

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["tutorial"]
collection = db["example"]

# Load data from the JSON file
with open('xml_data.json', 'r') as file:
    data = json.load(file)

# Insert the data into the collection
collection.insert_one(data)





InsertOneResult(ObjectId('662a55ba33b7e98b2b0ce78f'), acknowledged=True)

In [19]:
#4 : Get documents where authors key exist in the collection "CRUD_exercise".
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["TODO"]
collection = db["CRUD_exercise"]

documents_with_authors = collection.find({"authors": {"$exists": True}})
print(documents_with_authors)


In [9]:
#5 : Change the documents where x = 4 to x = 1.

collection.update_many({"x": 4}, {"$set": {"x": 1}})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [15]:
#6 : Find documents where author is not_mike and set author as real_mike.
collection.update_many({"author": "not_mike"}, {"$set": {"author": "real_mike"}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [11]:
#7 : Delete documents where author is real_mike
collection.delete_many({"author": "real_mike"})


DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [ ]:
### PARTIE 2 ###

In [20]:
#8 : create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

import pymongo
client = pymongo.MongoClient('localhost')
mydb = client["TODO"]
collection = mydb["CRUD_exercise_benchmark"]

import numpy as np

values = np.random.rand(500000, 4)
docs = [{"id": i, "values": values[i//2].tolist()} for i in range(0, 1000000, 2)]

collection.insert_many(docs)

import time

# No index
start_time = time.time()
docs = collection.find({"id": 100000})
end_time = time.time()
print("Temps sans index: ", end_time - start_time)

# Index
start_time = time.time()
docs = collection.find({"id": 100000}).hint("id_1")
end_time = time.time()
print("Temps avec index: ", end_time - start_time)


Temps sans index:  0.14007258415222168
Temps avec index:  0.0


In [21]:
#9 : 
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
random_db = client["random_db"]
random_collection = random_db["random_collection"]

doc = {"key": "value"}
random_collection.insert_one(doc)

tutorial_db = client["tutorial"]
tutorial_collection = tutorial_db["tutorial_collection"]
tutorial_collection.insert_many(random_collection.find())

InsertManyResult([ObjectId('662a575833b7e98b2b1488b5')], acknowledged=True)

In [ ]:
# 10 : 
# Inner join returns only the matching documents from both collections whereas outer join returns all documents 
# from both collections, including non-matching documents as null. The query seen during the course was an inner join.